In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA-2-RegExp

In [2]:
const { execSync } = await import('child_process');
console.log(execSync('npm install @viz-js/viz').toString());
import { DFA, State, Char, TransRel, dfa_2_regexp } from "./05-DFA-2-RegExp";
import * as fs from "fs";
import { instance } from "@viz-js/viz";


up to date, audited 13 packages in 2s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [3]:
const delta: TransRel = new Map<[State, Char], State>();
delta.set([1, "a"], 2);
delta.set([2, "b"], 3);
delta.set([3, "a"], 2);
const Q: Set<State> = new Set([1, 2, 3]);
const Sigma: Set<Char> = new Set(["a", "b"]);
const q0: State = 1;
const F: Set<State> = new Set([3]);
const A: DFA = { Q, Sigma, delta, q0, F };

In [4]:
console.log("✅ DFA Definition:");
console.log("  States:", Array.from(A.Q));
console.log("  Sigma:", Array.from(A.Sigma));
console.log("  Start State:", A.q0);
console.log("  Accepting States:", Array.from(A.F));
for (const [[q, sym], target] of A.delta.entries()) {
  console.log(`    δ(${q}, '${sym}') = ${target}`);
}

✅ DFA Definition:
  States: [ 1, 2, 3 ]
  Sigma: [ 'a', 'b' ]
  Start State: 1
  Accepting States: [ 3 ]
    δ(1, 'a') = 2
    δ(2, 'b') = 3
    δ(3, 'a') = 2


In [5]:
function dfa2dot(A: DFA): string {
  const lines: string[] = [];
  lines.push("digraph DFA {");
  lines.push("  rankdir=LR;");
  lines.push('  node [shape=circle, fontsize=12];');
  lines.push('  "" [shape=point];');
  lines.push(`  "" -> S${A.q0};`);

  // akzeptierende Zustände doppelt zeichnen
  for (const f of A.F) {
    lines.push(`  S${f} [shape=doublecircle];`);
  }

  // Übergänge
  for (const [[q, sym], target] of A.delta.entries()) {
    lines.push(`  S${q} -> S${target} [label="${sym}"];`);
  }

  lines.push("}");
  return lines.join("\n");
}

// --- Funktion: DFA anzeigen ------------------------------------------------
async function showDFA(A: DFA) {
  const dot = dfa2dot(A);
  const viz = await instance();
  const svg = await viz.renderString(dot, { format: "svg" });

  if (typeof display !== "undefined" && (display as any).html) {
    (display as any).html(svg);
  } else {
    console.log(svg);
  }
}

// --- Beispielaufruf --------------------------------------------------------
await showDFA(A);

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: DFA Pages: 1 -->
 
 
 DFA 
 
 
 
 
 
<!-- S1 -->
 
 S1 
 
 S1 
 
<!-- ->S1 -->
 
 ->S1 
 
 
 
<!-- S2 -->
 
 S2 
 
 S2 
 
<!-- S1->S2 -->
 
 S1->S2 
 
 
 a 
 
<!-- S3 -->
 
 S3 
 
 
 S3 
 
<!-- S3->S2 -->
 
 S3->S2 
 
 
 a 
 
<!-- S2->S3 -->
 
 S2->S3 
 
 
 b

In [6]:
const r = dfa_2_regexp(A);

// Hilfsfunktion: verschachtelte Arrays → lesbare Tupel-Notation
function regexp_to_tuple_string(r: any): string {
  if (r === 0) return "0";
  if (r === "ε" || r === "𝜀") return "𝜀";
  if (typeof r === "string") return `'${r}'`;

  if (Array.isArray(r)) {
    // Beispiel: [r1, '+', r2]
    if (r.length === 3) {
      const [a, op, b] = r;
      return `(${regexp_to_tuple_string(a)}, '${op}', ${regexp_to_tuple_string(b)})`;
    }
    // Beispiel: [r0, '*']
    if (r.length === 2 && r[1] === "*") {
      return `(${regexp_to_tuple_string(r[0])}, '*')`;
    }
  }
  return String(r);
}

// Ausgabe
console.log("\n✅ Generated Regular Expression (tuple style):");
console.log(regexp_to_tuple_string(r));


✅ Generated Regular Expression (tuple style):
(((0, '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', (𝜀, '+', 0))), '+', ((('a', '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', 'a')), '⋅', (((𝜀, '+', 0), '+', (('b', '⋅', ((𝜀, '+', 0), '*')), '⋅', 'a')), '*')), '⋅', ('b', '+', (('b', '⋅', ((𝜀, '+', 0), '*')), '⋅', (𝜀, '+', 0))))), '+', (((((𝜀, '+', 0), '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', 0)), '+', ((('a', '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', 'a')), '⋅', (((𝜀, '+', 0), '+', (('b', '⋅', ((𝜀, '+', 0), '*')), '⋅', 'a')), '*')), '⋅', (0, '+', (('b', '⋅', ((𝜀, '+', 0), '*')), '⋅', 0)))), '⋅', ((((𝜀, '+', 0), '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', 0)), '+', ((('a', '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', 'a')), '⋅', (((𝜀, '+', 0), '+', (('b', '⋅', ((𝜀, '+', 0), '*')), '⋅', 'a')), '*')), '⋅', (0, '+', (('b', '⋅', ((𝜀, '+', 0), '*')), '⋅', 0)))), '*')), '⋅', ((0, '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', (𝜀, '+', 0))), '+', ((('a', '+', ((0, '⋅', ((𝜀, '+', 0), '*')), '⋅', 'a')), '⋅', (((𝜀, '+', 0),

As this regular expression is nearly unreadable,  The notebook `Rewrite.ipynb` contains the definition of the function `simplify` that can be used to simplify this expression.

In [8]:
function simplify(r: any): any {
  // Basiscases --------------------------------------------------------------
  if (r === 0 || r === "0") return 0;
  if (r === "ε" || r === "𝜀") return "𝜀";
  if (typeof r === "string") return r;

  // Rekursion über Struktur -------------------------------------------------
  if (Array.isArray(r)) {
    // Fall: [r0, '*']
    if (r.length === 2 && r[1] === "*") {
      const inner = simplify(r[0]);

      // (0)* = 𝜀
      if (inner === 0) return "𝜀";
      // (𝜀)* = 𝜀
      if (inner === "𝜀") return "𝜀";

      // Sonderfall: (𝜀 + x)* → (x)*
      if (
        Array.isArray(inner) &&
        inner.length === 3 &&
        inner[0] === "𝜀" &&
        inner[1] === "+"
      ) {
        return [simplify(inner[2]), "*"];
      }

      return [inner, "*"];
    }

    // Fall: [r1, op, r2]
    if (r.length === 3) {
      const [r1, op, r2] = r.map(simplify);

      if (op === "+") {
        if (r1 === 0) return r2;
        if (r2 === 0) return r1;
        if (JSON.stringify(r1) === JSON.stringify(r2)) return r1;

        // Sonderfall: (𝜀 + x) → x*
        if (r1 === "𝜀") return r2;
        if (r2 === "𝜀") return r1;

        return [r1, "+", r2];
      }

      if (op === "⋅") {
        if (r1 === 0 || r2 === 0) return 0;
        if (r1 === "𝜀") return r2;
        if (r2 === "𝜀") return r1;
        return [r1, "⋅", r2];
      }
    }
  }

  return r;
}

// --- Test ------------------------------------------------------------------
const s = simplify(r);
console.log("\n✅ Simplified Regular Expression:");
console.log(regexp_to_tuple_string(s));


✅ Simplified Regular Expression:
(('a', '⋅', (('b', '⋅', 'a'), '*')), '⋅', 'b')


The function `regexp_2_string` takes a regular expression that is represented as a nested tuple and transforms it into a string.

In [9]:
// ---------------------------------------------------------------------------
// 06-Test-DFA-2-RegExp (Zelle 5)
// Wandelt eine verschachtelte RegExp-Struktur in eine Stringdarstellung um
// ---------------------------------------------------------------------------

function regexp_2_string(r: any): string {
  // leere Sprache
  if (r === 0) return "0";
  // leeres Wort
  if (r === "𝜀" || r === "ε") return "𝜀";
  // einzelnes Symbol
  if (typeof r === "string") return r;

  // Verkettung (⋅)
  if (Array.isArray(r) && r[1] === "⋅") {
    const [r1, _, r2] = r;
    return regexp_2_string(r1) + "⋅" + regexp_2_string(r2);
  }

  // Alternative (+)
  if (Array.isArray(r) && r[1] === "+") {
    const [r1, _, r2] = r;
    return "(" + regexp_2_string(r1) + "+" + regexp_2_string(r2) + ")";
  }

  // Stern (*)
  if (Array.isArray(r) && r[1] === "*") {
    const r0 = r[0];
    if (typeof r0 === "string") {
      return regexp_2_string(r0) + "*";
    } else {
      return "(" + regexp_2_string(r0) + ")*";
    }
  }

  throw new Error(`${r} is not a suitable regular expression`);
}

// --- Anwendung -------------------------------------------------------------
const resultString = regexp_2_string(s);
console.log("\n✅ Final Regular Expression String:");
console.log(resultString);


✅ Final Regular Expression String:
a⋅(b⋅a)*⋅b
